In [1]:
import os
from glob import glob
from utils import compute_mean_dice
import nibabel as nib
from scipy.spatial.distance import dice
import numpy as np
import itk
import SimpleITK as sitk
import scipy.ndimage
import scipy
import matplotlib.pyplot as plt
import scipy.interpolate
from transforms_dict import getRegistrationEvalInverseTransformForMRI, SaveTransformForMRI
from tqdm import tqdm
import monai
import subprocess

In [1]:
def interpolateWarp(warp, outwarp, step=8):
    mri = nib.load(warp)
    mri_data = mri.get_fdata()
    ptx = mri_data[step//2::step,step//2::step,step//2::step,:,:].squeeze()
    
    x = np.arange(step//2, mri_data.shape[0], step)
    y = np.arange(step//2, mri_data.shape[1], step)
    z = np.arange(step//2, mri_data.shape[2], step)   
    # xg, yg ,zg = np.meshgrid(x, y, z, indexing='ij', sparse=True)
    
    interp = scipy.interpolate.RegularGridInterpolator((x, y, z), ptx[:,:,:,:], 
                                                       bounds_error=False, 
                                                       fill_value=0, 
                                                       method="linear")
    
    xg = np.arange(0, mri_data.shape[0], 1)
    yg = np.arange(0, mri_data.shape[1], 1)
    zg = np.arange(0, mri_data.shape[2], 1)  
    X, Y, Z = np.meshgrid(xg, yg, zg, indexing='ij') 
    

    out = interp((X,Y,Z))
    
    out = out.reshape(mri_data.shape)
    
    affine = mri.affine
    header = mri.header 
    
    new_img = nib.Nifti1Image(out, affine, header)
    nib.save(new_img, outwarp)    

In [2]:
mridir = 'output/Feminad/DL_TestDef5/'
deformable_warps = sorted(glob(os.path.join(mridir, 'DeformableWarp', "*.nii.gz")))
for i, warp in enumerate(deformable_warps):
    outname = mridir + "DeformableWarpInterpolate/" + warp.split('/')[-1].split('.')[0] + ".nii.gz"
    interpolateWarp(deformable_warps[0], outname)    
    print(i, end='\r')

NameError: name 'glob' is not defined